In [1]:
import pandas as pd
cleaned_csv_path = "CSV/cleaned_data.csv" 
data = pd.read_csv(cleaned_csv_path)
print(data.head())


                                     id   item_date  quantity tons  \
0  EC06F063-9DF0-440C-8764-0B0C05A4F6AE  20210401.0      54.151139   
1  4E5F4B3D-DDDF-499D-AFDE-A3227EC49425  20210401.0     151.447449   
2  E140FF1B-2407-4C02-A0DD-780A093B1158  20210401.0     151.447449   
3  F8D507A0-9C62-4EFE-831E-33E1DA53BB50  20210401.0     151.447449   
4  4E1C4E78-152B-430A-8094-ADD889C9D0AD  20210401.0     151.447449   

     customer  country status item type  application  thickness   width  \
0  30156308.0     28.0    Won         W         10.0       2.00  1500.0   
1  30202938.0     25.0    Won         W         41.0       0.80  1210.0   
2  30153963.0     30.0    Won        WI         28.0       0.38   952.0   
3  30349574.0     32.0    Won         S         59.0       2.30  1317.0   
4  30211560.0     28.0    Won         W         10.0       4.00  1980.0   

              material_ref  product_ref  delivery date  selling_price  
0              DEQ1 S460MC   1670798778     20210701.0  

In [2]:

X_price = data[["thickness", "width", "quantity tons"]]  # Make sure these columns exist in your data
y_price = data["selling_price"]  # Label for price prediction


In [3]:
# Features for status classification
X_status = data[["thickness", "width", "quantity tons"]]  # Use the same features as price prediction
y_status = data["status"]  # Label for status classification


In [4]:
from sklearn.model_selection import train_test_split

# Split the data for price prediction
X_price_train, X_price_test, y_price_train, y_price_test = train_test_split(X_price, y_price, test_size=0.2, random_state=42)

# Split the data for status classification
X_status_train, X_status_test, y_status_train, y_status_test = train_test_split(X_status, y_status, test_size=0.2, random_state=42)


In [5]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the training data for price prediction
X_price_train_scaled = scaler.fit_transform(X_price_train)

# Transform the test data for price prediction
X_price_test_scaled = scaler.transform(X_price_test)

# Fit and transform the training data for status classification
X_status_train_scaled = scaler.fit_transform(X_status_train)

# Transform the test data for status classification
X_status_test_scaled = scaler.transform(X_status_test)


In [6]:
from sklearn.linear_model import LinearRegression, LogisticRegression

# Train the Linear Regression model for price prediction
price_model = LinearRegression()
price_model.fit(X_price_train_scaled, y_price_train)

# Train the Logistic Regression model for status classification
status_model = LogisticRegression(max_iter=1000)  # Increase max_iter if needed
status_model.fit(X_status_train_scaled, y_status_train)


LogisticRegression(max_iter=1000)

In [7]:
from sklearn.metrics import mean_absolute_error, accuracy_score, confusion_matrix, classification_report

# Predicting prices
y_price_pred = price_model.predict(X_price_test_scaled)

# Calculate MAE for price prediction
mae = mean_absolute_error(y_price_test, y_price_pred)
print(f'Mean Absolute Error for Price Prediction: {mae}')

# Predicting status
y_status_pred = status_model.predict(X_status_test_scaled)

# Calculate accuracy for status classification
accuracy = accuracy_score(y_status_test, y_status_pred)
print(f'Accuracy for Status Classification: {accuracy}')

# Confusion Matrix and Classification Report for status classification
confusion = confusion_matrix(y_status_test, y_status_pred)
report = classification_report(y_status_test, y_status_pred)

print("Confusion Matrix for Status Classification:")
print(confusion)
print("\nClassification Report for Status Classification:")
print(report)


Mean Absolute Error for Price Prediction: 154.4630443544827
Accuracy for Status Classification: 0.6390983651676116


C:\Users\Vishal\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vishal\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confusion Matrix for Status Classification:
[[    0     0     0     0     0     0     0   656]
 [    0    43     0     0     0     0     0  6726]
 [    0    13     0     0     0     0     0  3942]
 [    0     0     0     0     0     0     0     2]
 [    0     0     0     0     0     0     0    11]
 [    0     4     0     0     0     0     0   856]
 [    0     1     0     0     0     0     0   824]
 [    0    78     0     0     0     0     0 23178]]

Classification Report for Status Classification:
                 precision    recall  f1-score   support

          Draft       0.00      0.00      0.00       656
           Lost       0.31      0.01      0.01      6769
Not lost for AM       0.00      0.00      0.00      3955
      Offerable       0.00      0.00      0.00         2
        Offered       0.00      0.00      0.00        11
        Revised       0.00      0.00      0.00       860
 To be approved       0.00      0.00      0.00       825
            Won       0.64      1.00    

C:\Users\Vishal\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
from sklearn.model_selection import GridSearchCV

In [9]:
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Inverse of regularization strength
    'solver': ['liblinear', 'lbfgs', 'saga'],  # Different solvers
    'max_iter': [100, 500, 1000]  # Maximum number of iterations
}


In [10]:
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='accuracy', n_jobs=-1)


In [11]:
X = data[["thickness", "width", "quantity tons"]]  # Adjust based on your DataFrame's column names
y = data["status"]  # Label for classification



In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [13]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


In [15]:
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Inverse of regularization strength
    'solver': ['liblinear', 'lbfgs', 'saga'],  # Different solvers
    'max_iter': [100, 500, 1000]  # Maximum number of iterations
}


In [16]:
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='accuracy', n_jobs=-1)


In [19]:
import joblib
joblib.dump(model, 'logistic_regression_model.joblib')
print("Model saved as 'logistic_regression_model.joblib'")


NameError: name 'model' is not defined

In [20]:
# Save the Linear Regression model
joblib.dump(price_model, 'linear_regression_price_model.joblib')
print("Price prediction model saved as 'linear_regression_price_model.joblib'")

# Save the Logistic Regression model
joblib.dump(status_model, 'logistic_regression_status_model.joblib')
print("Status classification model saved as 'logistic_regression_status_model.joblib'")


Price prediction model saved as 'linear_regression_price_model.joblib'
Status classification model saved as 'logistic_regression_status_model.joblib'


In [22]:
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Sample dataset creation for price prediction
np.random.seed(42)  # For reproducibility
X_price = np.random.rand(100, 2)  # 100 samples, 2 features
y_price = X_price @ np.array([150, 300]) + 1000 + np.random.randn(100) * 10  # Linear relationship with noise

# Sample dataset creation for status classification
X_status = np.random.rand(100, 2)  # 100 samples, 2 features
y_status = (X_status[:, 0] + X_status[:, 1] > 1).astype(int)  # Simple binary classification

# Splitting the dataset into training and testing sets
X_price_train, X_price_test, y_price_train, y_price_test = train_test_split(X_price, y_price, test_size=0.2, random_state=42)
X_status_train, X_status_test, y_status_train, y_status_test = train_test_split(X_status, y_status, test_size=0.2, random_state=42)

# Scaling the features
scaler_price = StandardScaler()
X_price_train_scaled = scaler_price.fit_transform(X_price_train)
X_price_test_scaled = scaler_price.transform(X_price_test)

scaler_status = StandardScaler()
X_status_train_scaled = scaler_status.fit_transform(X_status_train)
X_status_test_scaled = scaler_status.transform(X_status_test)

# Train the Linear Regression model for price prediction
price_model = LinearRegression()
price_model.fit(X_price_train_scaled, y_price_train)

# Train the Logistic Regression model for status classification
status_model = LogisticRegression(max_iter=1000)
status_model.fit(X_status_train_scaled, y_status_train)

# Save the models
joblib.dump(price_model, 'linear_regression_price_model.joblib')
joblib.dump(status_model, 'logistic_regression_status_model.joblib')

# Load the models
loaded_price_model = joblib.load('linear_regression_price_model.joblib')
loaded_status_model = joblib.load('logistic_regression_status_model.joblib')

# Example prediction for price
sample_price_input = scaler_price.transform([[0.5, 0.5]])  # Example scaled input
predicted_price = loaded_price_model.predict(sample_price_input)
print("Predicted Price:", predicted_price)

# Example prediction for status
sample_status_input = scaler_status.transform([[0.7, 0.4]])  # Example scaled input
predicted_status = loaded_status_model.predict(sample_status_input)
print("Predicted Status:", predicted_status)


Predicted Price: [1225.8100823]
Predicted Status: [1]


In [23]:
from sklearn.linear_model import LinearRegression, LogisticRegression
import joblib
price_model = LinearRegression()
price_model.fit(X_price_train_scaled, y_price_train)
status_model = LogisticRegression(max_iter=1000)
status_model.fit(X_status_train_scaled, y_status_train)
# Save the models
joblib.dump(price_model, 'price_model.pkl')
joblib.dump(status_model, 'status_model.pkl')
print("Models saved successfully.")

Models saved successfully.
